# Глава 7: Създаване на чат приложения
## Бърз старт с Github Models API

Този тетрадка е адаптирана от [Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst), която включва тетрадки за достъп до услугите на [Azure OpenAI](notebook-azure-openai.ipynb).


# Общ преглед  
„Големите езикови модели са функции, които преобразуват текст в текст. Когато получат входен низ от текст, големият езиков модел се опитва да предвиди какъв текст ще последва“(1). Този „бърз старт“ наръчник ще запознае потребителите с основните концепции на LLM, основните изисквания за работа с AML, кратко въведение в дизайна на подсказки и няколко кратки примера за различни приложения.


## Съдържание

[Общ преглед](../../../../07-building-chat-applications/python)  
[Как да използвате OpenAI Service](../../../../07-building-chat-applications/python)  
[1. Създаване на вашия OpenAI Service](../../../../07-building-chat-applications/python)  
[2. Инсталация](../../../../07-building-chat-applications/python)  
[3. Данни за достъп](../../../../07-building-chat-applications/python)  

[Примери за употреба](../../../../07-building-chat-applications/python)  
[1. Обобщаване на текст](../../../../07-building-chat-applications/python)  
[2. Класифициране на текст](../../../../07-building-chat-applications/python)  
[3. Генериране на нови имена на продукти](../../../../07-building-chat-applications/python)  
[4. Фино настройване на класификатор](../../../../07-building-chat-applications/python)  

[Референции](../../../../07-building-chat-applications/python)


### Създайте първия си prompt  
Това кратко упражнение ще ви даде основни познания за подаване на prompts към модел в Github Models за проста задача „обобщение“.


**Стъпки**:  
1. Инсталирайте библиотеката `azure-ai-inference` във вашата python среда, ако все още не сте го направили.  
2. Заредете стандартните помощни библиотеки и настройте Github Models идентификационни данни.  
3. Изберете модел за вашата задача  
4. Създайте прост prompt за модела  
5. Изпратете заявката си към API-то на модела!


### 1. Инсталирайте `azure-ai-inference`


In [ ]:
%pip install azure-ai-inference

### 2. Импортирайте помощни библиотеки и създайте идентификационни данни


In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

### 3. Откриване на подходящия модел  
Моделите GPT-3.5-turbo или GPT-4 могат да разбират и генерират естествен език.


In [ ]:
# Select the General Purpose curie model for text
model_name = "gpt-4o"

## 4. Дизайн на подканите  

„Магията на големите езикови модели е, че като се обучават да минимизират грешката при предсказване върху огромни количества текст, моделите в крайна сметка научават концепции, които са полезни за тези предсказвания. Например, те научават концепции като“(1):

* как се пише правилно
* как работи граматиката
* как да перифразират
* как да отговарят на въпроси
* как да водят разговор
* как да пишат на много езици
* как да програмират
* и др.

#### Как да управляваме голям езиков модел  
„От всички входове към голям езиков модел, най-голямо влияние има текстовата подканa(1).

Големите езикови модели могат да бъдат подканени да генерират изход по няколко начина:

Инструкция: Кажете на модела какво искате
Довършване: Подтикнете модела да довърши началото на това, което искате
Демонстрация: Покажете на модела какво искате, като използвате:
Няколко примера в подканата
Стотици или хиляди примери в тренировъчен набор за фино настройване“



#### Има три основни насоки за създаване на подканa:

**Показвайте и обяснявайте**. Направете ясно какво искате – чрез инструкции, примери или комбинация от двете. Ако искате моделът да подреди списък по азбучен ред или да класифицира абзац по настроение, покажете му, че това очаквате.

**Осигурете качествени данни**. Ако се опитвате да създадете класификатор или да накарате модела да следва определен шаблон, уверете се, че има достатъчно примери. Прегледайте внимателно примерите си – моделът обикновено е достатъчно умен да разпознае елементарни правописни грешки и пак да ви даде отговор, но може и да приеме, че това е нарочно, което да повлияе на резултата.

**Проверете настройките си.** Настройките temperature и top_p контролират доколко детерминиран е моделът при генериране на отговор. Ако искате отговор, при който има само един правилен вариант, задайте ги по-ниски. Ако търсите по-разнообразни отговори, може да ги увеличите. Най-честата грешка при тези настройки е да се смята, че те управляват „умността“ или „креативността“ на модела.


Източник: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Обобщаване на текст  
#### Предизвикателство  
Обобщете текст, като добавите 'tl;dr:' в края на даден откъс. Обърнете внимание как моделът разбира как да изпълнява различни задачи без допълнителни инструкции. Можете да експериментирате с по-описателни подсказки от tl;dr, за да промените поведението на модела и да персонализирате обобщението, което получавате(3).  

Последни изследвания показват значителен напредък в много NLP задачи и бенчмаркове чрез предварително обучение върху голям корпус от текст, последвано от дообучаване за конкретна задача. Макар архитектурата обикновено да е независима от задачата, този метод все пак изисква специфични за задачата набори от данни с хиляди или десетки хиляди примери. За разлика от това, хората обикновено могат да изпълнят нова езикова задача само с няколко примера или с прости инструкции – нещо, с което настоящите NLP системи все още имат трудности. Тук показваме, че увеличаването на мащаба на езиковите модели значително подобрява независимото от задачата изпълнение с малко примери, като понякога дори достига конкурентност с предишни най-добри подходи за дообучаване.  

Tl;dr


# Упражнения за различни случаи на употреба  
1. Обобщаване на текст  
2. Класифициране на текст  
3. Генериране на нови имена на продукти


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Класифицирай текст  
#### Предизвикателство  
Класифицирай елементи в категории, които се задават по време на изпълнение. В следния пример предоставяме както категориите, така и текста за класифициране в подканата (*playground_reference).

Запитване от клиент: Здравейте, един от бутоните на клавиатурата на лаптопа ми се счупи наскоро и ще ми трябва заместител:

Класифицирана категория:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Генериране на нови имена за продукти
#### Предизвикателство
Създайте имена на продукти от примерни думи. Включваме в подсказката информация за продукта, за който ще генерираме имена. Даваме и подобен пример, за да покажем какъв модел очакваме. Зададената стойност на температурата е висока, за да се получат по-случайни и иновативни отговори.

Описание на продукта: Домашен уред за приготвяне на млечни шейкове
Ключови думи: бърз, здравословен, компактен.
Имена на продукти: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Описание на продукта: Чифт обувки, които пасват на всеки размер крак.
Ключови думи: адаптивен, пасва, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# Референции  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Примери](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Най-добри практики за фино настройване на GPT-3 за класифициране на текст](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# За повече помощ  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# Сътрудници
* [Chew-Yean Yam](https://www.linkedin.com/in/cyyam/)



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, имайте предвид, че автоматичните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия изходен език следва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Не носим отговорност за недоразумения или погрешни тълкувания, произтичащи от използването на този превод.
